In [14]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [4]:
path='/content/drive/MyDrive/Project Work/Colab Notebooks/suicide_detection_final_cleaned.csv'
data=pd.read_csv(path)

In [5]:
data.head()

,Unnamed: 0,text,class,clean_msg
0,3,Am I weird I don't get affected by compliments...,non-suicide,weird not get affected compliment coming someo...
1,4,Finally 2020 is almost over... So I can never ...,non-suicide,finally almost never hear bad year ever swear ...
2,8,i need helpjust help me im crying so hard,suicide,need help help cry hard
3,13,It ends tonight.I can’t do it anymore. \nI quit.,suicide,end tonight not anymore quit
4,19,I took the rest of my sleeping pills and my pa...,suicide,took rest sleeping pill painkiller not wait en...


In [5]:
!pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.6 MB/s eta 0:00:00


In [7]:
data['class']=data['class'].replace({'non-suicide': 0, 'suicide': 1})

In [8]:
data=data[['text','class']]

In [9]:
data.head()

,text,class
0,Am I weird I don't get affected by compliments...,0
1,Finally 2020 is almost over... So I can never ...,0
2,i need helpjust help me im crying so hard,1
3,It ends tonight.I can’t do it anymore. \nI quit.,1
4,I took the rest of my sleeping pills and my pa...,1


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
import torch
from transformers import TrainingArguments,Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [11]:

torch.device("cuda")
print(torch.cuda.current_device())

0


In [7]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
chec=["not feeling  well","getting lost in path of life"]
tokenizer(chec,padding=True,truncation=True,max_length=100)

{'input_ids': [[101, 2025, 3110, 2092, 102, 0, 0, 0], [101, 2893, 2439, 1999, 4130, 1997, 2166, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [15]:
X=list(data['text'])
y=list(data['class'])
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,stratify=y)


In [16]:
X_train_tokenized=tokenizer(X_train,padding='max_length',truncation=True)
X_test_tokenized=tokenizer(X_test,padding='max_length',truncation=True)

In [17]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [18]:
#creating Custom torch dataset as written in Tranformers docs
class Dataset(torch.utils.data.Dataset):
    def __init__(self,encodings,labels=None):
        self.encodings=encodings
        self.labels=labels
    def __getitem__(self,idx):
        item={key:torch.tensor(val[idx])for key,val in self.encodings.items()}
        if self.labels:
            item["labels"]=torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.encodings["input_ids"])

In [19]:
train_dataset=Dataset(X_train_tokenized,y_train)
test_dataset=Dataset(X_test_tokenized,y_test)

In [20]:
train_dataset[4]

{'input_ids': tensor([  101,  2054,  2015,  1996,  2087, 16436,  2518,  2008,  2038,  3047,
          2000,  2017,  2005,  2033,  2009,  2001,  2043,  1045,  2001,  2012,
          2082,  1010,  2057,  2020,  5604,  2061,  1996,  2465,  2003,  2428,
          4251,  1997,  2607,  1998,  2061,  1045,  3062,  6680,  1998,  2292,
          2041,  1037,  5189,  2521,  2102,  1045,  2246,  2039,  1998,  3071,
          2001,  2559,  2875,  2073,  1045,  2001,  3564,  2021,  1045,  3092,
          2039, 24114,  2009,  2006,  1996,  3124,  1999,  2392,  1997,  2033,
          2029,  3071,  2059, 15912,  2098,  2006,  1010,  3374, 22953,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [21]:
def compute_metrics(p):
    print(type(p))
    pred,labels=p
    pred=np.argmax(pred,axis=1)
    
    accuracy=accuracy_score(y_true=labels,y_pred=pred)
    recall=recall_score(y_true=labels,y_pred=pred)
    precision=precision_score(y_true=labels,y_pred=pred)
    f1=f1_score(y_true=labels,y_pred=pred)
    
    return {"accuracy":accuracy,'precision':precision,'recall':recall,'f1_score':f1}

In [22]:
args=TrainingArguments(output_dir='./output',overwrite_output_dir = True,learning_rate=1e-5,num_train_epochs=1,per_device_train_batch_size=6,)
trainer=Trainer(model=model,args=args,train_dataset=train_dataset,eval_dataset=test_dataset,compute_metrics=compute_metrics)
# trainer=Trainer(model=model,args=args,train_dataset=train_dataset,compute_metrics=compute_metrics)

In [23]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.309400
1000,0.227200
1500,0.205200
2000,0.144700
2500,0.173900
3000,0.169300
3500,0.149000
4000,0.151600
4500,0.129400
5000,0.126300


TrainOutput(global_step=16514, training_loss=0.13633486145241752, metrics={'train_runtime': 9237.5076, 'train_samples_per_second': 10.726, 'train_steps_per_second': 1.788, 'total_flos': 2.606956958717952e+16, 'train_loss': 0.13633486145241752, 'epoch': 1.0})

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/bert_saved_model')

NameError: ignored

In [25]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.10597800463438034,
 'eval_accuracy': 0.9760809010536514,
 'eval_precision': 0.9663141239173884,
 'eval_recall': 0.9678455250646426,
 'eval_f1_score': 0.9670792182356127,
 'eval_runtime': 2001.2432,
 'eval_samples_per_second': 33.007,
 'eval_steps_per_second': 4.126,
 'epoch': 1.0}

In [11]:
model_2=BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/bert_saved_model')


In [12]:
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [15]:
np.set_printoptions(suppress=True)

In [30]:
text="Life is too short to commit suicide Be patient you will die"
inputs=tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')
outputs=model_2(**inputs)
prediction=torch.nn.functional.softmax(outputs.logits,dim=-1)
prediction=prediction.cpu().detach().numpy()
prediction

array([[0.9927048 , 0.00729515]], dtype=float32)